In [1]:
import os
import sys
import glob
import shutil
from pathlib import Path
from tempfile import TemporaryDirectory
from concurrent.futures import ProcessPoolExecutor
from multiprocessing import Pool
import pandas as pd

In [2]:
sys.path.append('../src')

In [3]:
from run import run_plasmidfinder

In [6]:
database = '/media/GenomicResearch/Tools/CGE/plasmidfinder_db'

In [12]:
dirpath = Path("/media/NAS/Central_Lab_Storage/MiSeq/Salmonella_enterica/Contigs")
outpath = Path("/media/NAS/Central_Lab_Storage/MiSeq/Salmonella_enterica/Analysis")

In [19]:
with Pool(24) as p:
    try:
        for filepath in dirpath.iterdir():
            outdir = outpath/filepath.stem/'plasmidfinder'
            if outdir.exists() is False:
                p.apply_async(run_plasmidfinder, (filepath, outdir, database))
        p.close()
        p.join()
    except:
        p.terminate()

In [24]:
def plasmidfinder_summary(filepath):
    df = pd.read_csv(filepath, sep='\t')
    return df['Plasmid'].sort_values().str.cat(sep=', ')

In [20]:
dirpath = Path('/media/NAS/Central_Lab_Storage/MiSeq/Salmonella_enterica/Analysis')

In [25]:
summaries = dict()
for i in dirpath.iterdir():
    filepath = i/'plasmidfinder'/'results_tab.tsv'
    summary = plasmidfinder_summary(filepath)
    summaries[i.name] = summary

In [29]:
s = pd.Series(summaries)
s.name = 'plasmid_type'
s.index.name = 'Key'

In [32]:
s.to_csv('/media/NAS/Central_Lab_Storage/MiSeq/Salmonella_enterica/plasmidfiner.tsv', sep='\t')

In [7]:
run_plasmidfinder('/media/Central_Lab_Storage/MinION/mNGS/20211214_ICU11-15/denovo/barcode01+02+03/contigs.fasta',
                  '/media/Central_Lab_Storage/MinION/mNGS/20211214_ICU11-15/plasmidfinder/barcode01+02+03', database)